# 🔥 GRU Model Training: Emotion Classification

This notebook trains a GRU (Gated Recurrent Unit) model on preprocessed emotion data.


## 📚 Import Libraries


In [10]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Bidirectional, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')


## 📂 Load Preprocessed Data


In [11]:
# Load preprocessed training and validation data
train_df = pd.read_pickle('./data/train_preprocessed.pkl')
val_df = pd.read_pickle('./data/validation_preprocessed.pkl')

print(f"Training data shape: {train_df.shape}")
print(f"Validation data shape: {val_df.shape}")
print(f"\nColumns: {train_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(train_df.head())


Training data shape: (15999, 2)
Validation data shape: (2000, 2)

Columns: ['Text', 'Label']

First few rows:
                                                Text  Label
0                              didnt feel humiliated      0
1  go feeling hopeless damned hopeful around some...      0
2          im grabbing minute post feel greedy wrong      3
3  ever feeling nostalgic fireplace know still pr...      2
4                                    feeling grouchy      3


In [12]:
train_df.head(10)

,Text,Label
0,didnt feel humiliated,0
1,go feeling hopeless damned hopeful around some...,0
2,im grabbing minute post feel greedy wrong,3
3,ever feeling nostalgic fireplace know still pr...,2
4,feeling grouchy,3
5,ive feeling little burdened lately wasnt sure,0
6,ive taking milligrams times recommended amount...,5
7,feel confused life teenager jaded year old man,4
8,petronas years feel petronas performed well ma...,1
9,feel romantic,2


## 📊 Prepare Data

Split the data into features (X) and labels (y).


In [13]:
#  Prepare X and y
X_train = train_df['Text']
y_train = train_df['Label']
X_val = val_df['Text']
y_val = val_df['Label']

print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"\nLabel distribution in training set:")
print(y_train.value_counts().sort_index())


Training samples: 15999
Validation samples: 2000

Label distribution in training set:
Label
0    4666
1    5361
2    1304
3    2159
4    1937
5     572
Name: count, dtype: int64


## 🔠 Tokenization

Convert text to sequences of integers.


In [14]:
# Initialize tokenizer
tokenizer = Tokenizer(num_words=60000)
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_val_sequences = tokenizer.texts_to_sequences(X_val)

# Find maximum sequence length
maxlen = max(len(tokens) for tokens in X_train_sequences)
print(f"Maximum sequence length: {maxlen}")

# Pad sequences
X_train_padded = pad_sequences(X_train_sequences, maxlen=maxlen, padding='post')
X_val_padded = pad_sequences(X_val_sequences, maxlen=maxlen, padding='post')

print(f"\nX_train_padded shape: {X_train_padded.shape}")
print(f"X_val_padded shape: {X_val_padded.shape}")

# Calculate input size for embedding layer
input_size = np.max(X_train_padded) + 1
print(f"Vocabulary size (input_size): {input_size}")


Maximum sequence length: 35

X_train_padded shape: (15999, 35)
X_val_padded shape: (2000, 35)
Vocabulary size (input_size): 15064


## 🛠️ Build GRU Model

**Model Architecture:**
1. **Embedding Layer**: Converts word indices to dense vectors
2. **Bidirectional GRU**: Processes sequences in both directions
3. **Batch Normalization**: Normalizes activations for stable training
4. **Dropout**: Prevents overfitting (50% dropout rate)
5. **Dense Layer**: Fully connected layer with ReLU activation
6. **Output Layer**: 6 units with softmax for emotion classification


In [15]:
# Define the GRU model
model = Sequential()

# Embedding layer
model.add(Embedding(input_dim=input_size, output_dim=100, input_shape=(maxlen,)))

# Bidirectional GRU layer with 128 units
model.add(Bidirectional(GRU(128)))

# Batch normalization
model.add(BatchNormalization())

# Dropout for regularization
model.add(Dropout(0.5))

# Dense layer with ReLU activation
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer with 6 units (6 emotions) and softmax activation
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 35, 100)        │     1,506,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 256)            │       176,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,700,902 (6.49 MB)

 Trainable params: 1,700,390 (6.49 MB)

 Non-trainable params: 512 (2.00 KB)

## 🚀 Train the Model

Train with early stopping to prevent overfitting.


In [ ]:
# Train the model
history = model.fit(
    X_train_padded, y_train,
    epochs=15,
    batch_size=32,
    validation_data=(X_val_padded, y_val),
    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)]
)


Epoch 1/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.6147 - loss: 1.0496 - val_accuracy: 0.4205 - val_loss: 2.1398
Epoch 2/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - accuracy: 0.8886 - loss: 0.3536 - val_accuracy: 0.1245 - val_loss: 9.4992
Epoch 3/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.9355 - loss: 0.2039 - val_accuracy: 0.5950 - val_loss: 1.9400
Epoch 4/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 21s 41ms/step - accuracy: 0.9489 - loss: 0.1563 - val_accuracy: 0.8655 - val_loss: 0.6185
Epoch 5/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 35s 70ms/step - accuracy: 0.9564 - loss: 0.1347 - val_accuracy: 0.8430 - val_loss: 0.5376
Epoch 6/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 19s 39ms/step - accuracy: 0.9645 - loss: 0.1137 - val_accuracy: 0.8120 - val_loss: 0.8112
Epoch 7/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 17s 35ms/step - accuracy: 0.9687 - loss: 0.0971 - val_accuracy: 0.8795 - val_loss: 0.4603
Epoch 8/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 17s 34ms/step - accuracy: 0.9741 - loss: 0.0799 - 

## 📊 Visualize Training Progress


In [ ]:
# Get the epoch with the highest validation accuracy
best_epoch = history.history['val_accuracy'].index(max(history.history['val_accuracy'])) + 1

# Create a subplot with 1 row and 2 columns
fig, axs = plt.subplots(1, 2, figsize=(16, 5))

# Plot training and validation accuracy
axs[0].plot(history.history['accuracy'], label='Training Accuracy', color='blue')
axs[0].plot(history.history['val_accuracy'], label='Validation Accuracy', color='red')
axs[0].scatter(best_epoch - 1, history.history['val_accuracy'][best_epoch - 1], 
               color='green', label=f'Best Epoch: {best_epoch}')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Accuracy')
axs[0].set_title('Training and Validation Accuracy')
axs[0].legend()
axs[0].grid(True)

# Plot training and validation loss
axs[1].plot(history.history['loss'], label='Training Loss', color='blue')
axs[1].plot(history.history['val_loss'], label='Validation Loss', color='red')
axs[1].scatter(best_epoch - 1, history.history['val_loss'][best_epoch - 1], 
               color='green', label=f'Best Epoch: {best_epoch}')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Loss')
axs[1].set_title('Training and Validation Loss')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()


## 📈 Evaluate Model Performance


In [ ]:
# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(X_val_padded, y_val)
print(f"\nValidation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")


## 🎯 Confusion Matrix


In [ ]:
# Get predictions
y_val_pred = model.predict(X_val_padded)
y_val_pred = np.argmax(y_val_pred, axis=1)

# Create confusion matrix
cm = confusion_matrix(y_val, y_val_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'],
            yticklabels=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'])
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('GRU Model - Confusion Matrix')
plt.tight_layout()
plt.show()


## 📝 Classification Report


In [ ]:
# Print classification report
emotion_names = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, target_names=emotion_names))


## 💾 Save Model and Tokenizer

Save the trained model and tokenizer for later use.


In [ ]:
# Save the model
model.save('./data/gru_model.keras')
print("✅ Model saved to: ./data/gru_model.keras")

# Save the tokenizer
with open('./data/gru_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
print("✅ Tokenizer saved to: ./data/gru_tokenizer.pkl")

# Save training metadata
metadata = {
    'maxlen': maxlen,
    'input_size': input_size,
    'label_mapping': label_mapping,
    'val_accuracy': val_accuracy,
    'val_loss': val_loss,
    'best_epoch': best_epoch
}

with open('./data/gru_metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)
print("✅ Metadata saved to: ./data/gru_metadata.pkl")

print(f"\n{'='*60}")
print("GRU MODEL TRAINING COMPLETE!")
print(f"{'='*60}")
print(f"Final Validation Accuracy: {val_accuracy:.4f}")
print(f"Final Validation Loss: {val_loss:.4f}")
